# Classifying hand gestures with LSTM and Keras

In [1]:
# Gather data set into paths and class labels
import glob, os
import re

paths = []
labels = []
for path, subdirs, files in os.walk('DigitData'):
    for name in files:
        paths.append(os.path.join(path, name))
        labels.append(re.split(r'(\d+)', name)[0])
print(len(paths), len(labels))

360 360


In [2]:
# Verify we have 15 unique classes
myset = set(labels)
num_classes = len(myset)
print(myset, num_classes)
import pandas as pd
#Get shape if input data
df = pd.read_csv(paths[0], header=None, nrows=20000)
timestep = df.shape[0]
num_channels = df.shape[1]

{'T_T', 'I_M', 'HC_', 'M_M', 'IMR', 'T_M', 'T_R', 'M_R', 'I_I', 'R_R', 'T_I', 'R_L', 'MRL', 'L_L', 'T_L'} 15


In [3]:
#Dealing with x data
import numpy as np
#Initialize x data
x_data = np.empty((len(paths), timestep, num_channels), dtype=np.float32)
#Load dat to tensor
for i, path in enumerate(paths):
    data = pd.read_csv(path, header=None, nrows=20000)
    x_data[i,:,:] = data.values
import scipy.signal as sg
decimated = sg.decimate(x_data, axis = 1, q =20)

In [9]:
#Dealing with y data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#Change class label to one-hot vectors 
le = LabelEncoder()
one_hot = OneHotEncoder(sparse = False)

#Convert to ints
le.fit(list(myset))
int_labels = le.transform(labels)
int_labels

array([ 4,  9,  0,  1, 10, 14,  6,  3, 10, 11,  8,  3, 12,  5,  8,  1,  6,
       13,  5, 11, 12,  0, 12,  0,  3,  2,  5,  9,  9,  8,  1,  2, 14, 14,
       11,  4,  7, 13,  4,  2,  6, 10, 13,  7,  7,  4,  9,  0,  1, 10, 14,
        6,  3, 10, 11,  8,  3, 12,  5,  8,  1,  6, 13,  5, 11, 12,  0, 12,
        0,  3,  2,  5,  9,  9,  8,  1,  2, 14, 14, 11,  4,  7, 13,  4,  2,
        6, 10, 13,  7,  7,  4,  9,  0,  1, 10, 14,  6,  3, 10, 11,  8,  3,
       12,  5,  8,  1,  6, 13,  5, 11, 12,  0, 12,  0,  3,  2,  5,  9,  9,
        8,  1,  2, 14, 14, 11,  4,  7, 13,  4,  2,  6, 10, 13,  7,  7,  4,
        9,  0,  1, 10, 14,  6,  3, 10, 11,  8,  3, 12,  5,  8,  1,  6, 13,
        5, 11, 12,  0, 12,  0,  3,  2,  5,  9,  9,  8,  1,  2, 14, 14, 11,
        4,  7, 13,  4,  2,  6, 10, 13,  7,  7,  4,  9,  0,  1, 10, 14,  6,
        3, 10, 11,  8,  3, 12,  5,  8,  1,  6, 13,  5, 11, 12,  0, 12,  0,
        3,  2,  5,  9,  9,  8,  1,  2, 14, 14, 11,  4,  7, 13,  4,  2,  6,
       10, 13,  7,  7,  4

In [10]:
#Splitting data to train test val split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(decimated,int_labels, test_size = 0.2, random_state=40, 
                                                   stratify = int_labels)

In [11]:
#Normalize the data
channel_means = np.mean(X_train, axis = (0,1)).reshape(1,8)
channel_std = np.std(X_train, axis = (0,1)).reshape(1,8)
X_train -= channel_means
X_train /= channel_std

In [12]:
#Split train into train-val
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state=40, 
                                                 stratify = y_train)

In [17]:
#Set up Keras model
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(256,
              input_shape = (1000, num_channels), return_sequences = True))
model.add(LSTM(256, return_sequences = True))
model.add(LSTM(256))
model.add(Dense(num_classes, activation = 'softmax'))

In [21]:
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer= 'adam',
             metrics = ['accuracy'])

In [22]:
model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_val,y_val))

Train on 230 samples, validate on 58 samples
Epoch 1/50
230/230 [==============================] - 30s 132ms/step - loss: 2.6678 - acc: 0.1130 - val_loss: 2.6908 - val_acc: 0.1207
Epoch 2/50
230/230 [==============================] - 28s 121ms/step - loss: 2.5395 - acc: 0.0957 - val_loss: 2.5798 - val_acc: 0.1034
Epoch 3/50
230/230 [==============================] - 29s 126ms/step - loss: 2.4489 - acc: 0.1435 - val_loss: 2.5075 - val_acc: 0.1034
Epoch 4/50
230/230 [==============================] - 29s 127ms/step - loss: 2.3965 - acc: 0.1652 - val_loss: 2.4034 - val_acc: 0.1552
Epoch 5/50
230/230 [==============================] - 29s 127ms/step - loss: 2.3639 - acc: 0.1478 - val_loss: 2.4268 - val_acc: 0.1379
Epoch 6/50
230/230 [==============================] - 29s 128ms/step - loss: 2.2999 - acc: 0.1696 - val_loss: 2.3633 - val_acc: 0.1552
Epoch 7/50
230/230 [==============================] - 30s 129ms/step - loss: 2.2023 - acc: 0.1913 - val_loss: 2.3388 - val_acc: 0.2069
Epoch 8/50

(230, 15)

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.